# Working with functions in Azure OpenAI
This notebook shows how to use the Chat Completions API in combination with functions to extend the current capabilities of GPT models. GPT models, do not inherently support real-time interaction with external systems, databases, or files. However, functions can be used to do so.

Overview: <br>
`tools` (previously called `functions`) is an optional parameter in the Chat Completion API which can be used to provide function specifications. This allows models to generate function arguments for the specifications provided by the user. 

Note: The API will not execute any function calls. Executing function calls using the outputed argments must be done by developers. 

## Setup

In [5]:
# if needed, install and/or upgrade to the latest version of the OpenAI Python library
# %pip install --upgrade openai

In [6]:
import os
import json
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from openai import AzureOpenAI
from dotenv import load_dotenv
import os

load_dotenv(override=True)

token_provider = get_bearer_token_provider(
    DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default"
)

# Initialize the Azure OpenAI client
client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
    #api_key=os.getenv("AZURE_OPENAI_KEY"),  
    azure_ad_token_provider=token_provider,
    api_version="2025-02-01-preview"
)

model_name = "gpt-4o" #os.environ["MODEL_NAME"] # You need to ensure the version of the model you are using supports the function calling feature

## 1.0 Test functions

This code calls the model with the user query and the set of functions defined in the functions parameter. The model then can choose if it calls a function. If a function is called, the content will be in a strigified JSON object. The function call that should be made and arguments are location in:  response[`choices`][0][`function_call`].

In [7]:
def get_function_call(messages, tool_choice="auto"):
    # Define the functions to use
    tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        },
    ]

    # Call the model with the user query (messages) and the functions defined in the functions parameter
    response = client.chat.completions.create(
        model=model_name,
        messages=messages,
        tools=tools,
        tool_choice=tool_choice,
    )

    return response

### Forcing the use of a specific function or no function
By changing the value of the `tool_choice` parameter you can allow the model to decide what function to use, force the model to use a specific function, or force the model to use no function.

In [8]:
first_message = [
    {"role": "user", "content": "What's the weather like in San Francisco?"}
]
# 'auto' : Let the model decide what function to call
print("Let the model decide what function to call:")
print(get_function_call(first_message, "auto").choices[0].message)

# 'none' : Don't call any function
print("Don't call any function:")
print(get_function_call(first_message, "none").choices[0].message)

# force a specific function call
print("Force a specific function call:")
print(
    get_function_call(
        first_message,
        tool_choice={"type": "function", "function": {"name": "get_current_weather"}},
    )
    .choices[0]
    .message
)

Let the model decide what function to call:
ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_hmGkqqfH8Nb4vms0LTROp4qe', function=Function(arguments='{"location":"San Francisco, CA"}', name='get_current_weather'), type='function')])
Don't call any function:
ChatCompletionMessage(content='Let me check the current weather in San Francisco for you. Hold on a moment!', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None)
Force a specific function call:
ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_jp1rzwLmSTnhA5vajhvlwWKt', function=Function(arguments='{"location":"San Francisco, CA","unit":"fahrenheit"}', name='get_current_weather'), type='function')])


## 2.0 Defining functions
Now that we know how to work with functions, let's define some functions in code so that we can walk through the process of using functions end to end.

### Function #1: Get current time

In [9]:
import pytz
from datetime import datetime


def get_current_time(location):
    try:
        # Get the timezone for the city
        timezone = pytz.timezone(location)

        # Get the current time in the timezone
        now = datetime.now(timezone)
        current_time = now.strftime("%I:%M:%S %p")

        return current_time
    except:
        return "Sorry, I couldn't find the timezone for that location."

In [10]:
get_current_time("America/New_York")

'04:27:28 PM'

### Function #2: Get stock market data
For simplicity, we're just hard coding some stock market data but you could easily edit the code to call out to an API to retrieve real-time data.

In [11]:
import pandas as pd
import json


def get_stock_market_data(index):
    available_indices = [
        "S&P 500",
        "NASDAQ Composite",
        "Dow Jones Industrial Average",
        "Financial Times Stock Exchange 100 Index",
    ]

    if index not in available_indices:
        return "Invalid index. Please choose from 'S&P 500', 'NASDAQ Composite', 'Dow Jones Industrial Average', 'Financial Times Stock Exchange 100 Index'."

    # Read the CSV file
    data = pd.read_csv("stock_data.csv")

    # Filter data for the given index
    data_filtered = data[data["Index"] == index]

    # Remove 'Index' column
    data_filtered = data_filtered.drop(columns=["Index"])

    # Convert the DataFrame into a dictionary
    hist_dict = data_filtered.to_dict()

    for key, value_dict in hist_dict.items():
        hist_dict[key] = {k: v for k, v in value_dict.items()}

    return json.dumps(hist_dict)

In [12]:
print(get_stock_market_data("NASDAQ Composite"))

{"Date": {"2": "2023-07-12", "3": "2023-07-13"}, "Open": {"2": 14000.65, "3": 14100.11}, "High": {"2": 14200.06, "3": 14250.0}, "Low": {"2": 13800.08, "3": 14000.67}, "Close": {"2": 14100.44, "3": 14050.81}, "Volume": {"2": 4000000, "3": 4200000}}


### Function #3: Calculator 

In [13]:
import math


def calculator(num1, num2, operator):
    if operator == "+":
        return str(num1 + num2)
    elif operator == "-":
        return str(num1 - num2)
    elif operator == "*":
        return str(num1 * num2)
    elif operator == "/":
        return str(num1 / num2)
    elif operator == "**":
        return str(num1**num2)
    elif operator == "sqrt":
        return str(math.sqrt(num1))
    else:
        return "Invalid operator"

In [14]:
print(calculator(5, 5, "+"))

10


## 3.0 Calling a function using GPT

Steps for Function Calling: 

1. Call the model with the user query and a set of functions defined in the functions parameter.
2. The model can choose to call a function; if so, the content will be a stringified JSON object adhering to your custom schema (note: the model may generate invalid JSON or hallucinate parameters).
3. Parse the string into JSON in your code, and call your function with the provided arguments if they exist.
4. Call the model again by appending the function response as a new message, and let the model summarize the results back to the user.

### 3.1 Describe the functions so that the model knows how to call them

In [15]:
from azure.identity import DefaultAzureCredential
from sqlalchemy import create_engine
from sqlalchemy import Index, create_engine, select, text
from sqlalchemy.orm import DeclarativeBase, Mapped, Session, mapped_column
import urllib

from dotenv import load_dotenv
import os
import json

load_dotenv(override=True)

def query_db(query: str) -> str:
        """
        print("Authenticating to Azure Database for PostgreSQL using Azure Identity...")
        POSTGRES_HOST = os.environ["POSTGRES_HOST"]
        POSTGRES_USERNAME = os.environ["POSTGRES_USERNAME"]
        POSTGRES_DATABASE = os.environ["POSTGRES_DATABASE"]
        POSTGRES_SSL = os.environ.get("POSTGRES_SSL")

        azure_credential = DefaultAzureCredential()
        token = azure_credential.get_token("https://ossrdbms-aad.database.windows.net/.default")
        POSTGRES_PASSWORD = token.token

        DATABASE_URI = f"postgresql://{POSTGRES_USERNAME}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}/{POSTGRES_DATABASE}"
        # Specify SSL mode if needed
        DATABASE_URI += f"?sslmode={POSTGRES_SSL}"

        engine = create_engine(DATABASE_URI, echo=False)

        # Run query
        results = []
        with Session(engine) as session:
                most_similars = session.execute(text(query)) #.scalars()
                #print(f"Most similar recipes to 'chicken and peanut':")
                #print(f"--------------------------------------------------")
                for Recipe in most_similars:
                        results.append({"recipe": Recipe[1], "score": Recipe[2]})
                        #print(f"INPUT: {Recipe[0]}: \nOUTPUT: {Recipe[1]} SCORE: ({Recipe[2]})")
                        #print(f"--------------------------------------------------")
        
        """

        #GEnerate dummy resuls
        results = [
            {"recipe": "Chicken Curry", "score": 0.95},
            {"recipe": "Peanut Butter Chicken", "score": 0.90},
            {"recipe": "Chicken Satay", "score": 0.85},
        ]
        return json.dumps(results)

In [16]:
query_db("test")


'[{"recipe": "Chicken Curry", "score": 0.95}, {"recipe": "Peanut Butter Chicken", "score": 0.9}, {"recipe": "Chicken Satay", "score": 0.85}]'

In [17]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_time",
            "description": "Get the current time in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The location name. The pytz is used to get the timezone for that location. Location names should be in a format like America/New_York, Asia/Bangkok, Europe/London",
                    }
                },
                "required": ["location"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "query_db",
            "description": "Execute SQL query",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "query to be executed in SQL database",
                    },
                },
                "required": ["query"],
            },
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_stock_market_data",
            "description": "Get the stock market data for a given index",
            "parameters": {
                "type": "object",
                "properties": {
                    "index": {
                        "type": "string",
                        "enum": [
                            "S&P 500",
                            "NASDAQ Composite",
                            "Dow Jones Industrial Average",
                            "Financial Times Stock Exchange 100 Index",
                        ],
                    },
                },
                "required": ["index"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "calculator",
            "description": "A simple calculator used to perform basic arithmetic operations",
            "parameters": {
                "type": "object",
                "properties": {
                    "num1": {"type": "number"},
                    "num2": {"type": "number"},
                    "operator": {
                        "type": "string",
                        "enum": ["+", "-", "*", "/", "**", "sqrt"],
                    },
                },
                "required": ["num1", "num2", "operator"],
            },
        },
    },
]

available_functions = {
    "get_current_time": get_current_time,
    "query_db": query_db,
    "get_stock_market_data": get_stock_market_data,
    "calculator": calculator,
}

### 3.2 Define a helper function to validate the function call
It's possible that the models could generate incorrect function calls so it's important to validate the calls. Here we define a simple helper function to validate the function call although you could apply more complex validation for your use case.

In [18]:
import inspect


# helper method used to check if the correct arguments are provided to a function
def check_args(function, args):
    sig = inspect.signature(function)
    params = sig.parameters

    # Check if there are extra arguments
    for name in args:
        if name not in params:
            return False
    # Check if the required arguments are provided
    for name, param in params.items():
        if param.default is param.empty and name not in args:
            return False

    return True

In [19]:
def run_conversation(messages, tools, available_functions):
    # Step 1: send the conversation and available functions to GPT
    response = client.chat.completions.create(
        model=model_name,
        messages=messages,
        tools=tools,
        tool_choice="auto",
    )

    response_message = response.choices[0].message

    # Step 2: check if GPT wanted to call a function
    if response_message.tool_calls:
        print("Recommended Function call:")
        print(response_message.tool_calls[0])
        print()

        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors

        function_name = response_message.tool_calls[0].function.name

        # verify function exists
        if function_name not in available_functions:
            return "Function " + function_name + " does not exist"
        function_to_call = available_functions[function_name]

        # verify function has correct number of arguments
        function_args = json.loads(response_message.tool_calls[0].function.arguments)
        if check_args(function_to_call, function_args) is False:
            return "Invalid number of arguments for function: " + function_name
        function_response = function_to_call(**function_args)

        print("Output of function call:")
        print(function_response)
        print()

        # Step 4: send the info on the function call and function response to GPT

        # adding assistant response to messages
        messages.append(
            {
                "role": response_message.role,
                "function_call": {
                    "name": response_message.tool_calls[0].function.name,
                    "arguments": response_message.tool_calls[0].function.arguments,
                },
                "content": None,
            }
        )

        # adding function response to messages
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response

        print("Messages in second request:")
        for message in messages:
            print(message)
        print()

        second_response = client.chat.completions.create(
            messages=messages,
            model=model_name,
            tools=tools,
            tool_choice="auto",
        )  # get a new response from GPT where it can see the function response

        return second_response

In [20]:
messages = [{"role": "user", "content": "What time is it in New York?"}]
assistant_response = run_conversation(messages, tools, available_functions)
print(assistant_response.choices[0].message)

Recommended Function call:
ChatCompletionMessageToolCall(id='call_FSZViPvOWW47FGpPyuaWtsqt', function=Function(arguments='{"location":"America/New_York"}', name='get_current_time'), type='function')

Output of function call:
04:27:33 PM

Messages in second request:
{'role': 'user', 'content': 'What time is it in New York?'}
{'role': 'assistant', 'function_call': {'name': 'get_current_time', 'arguments': '{"location":"America/New_York"}'}, 'content': None}
{'role': 'function', 'name': 'get_current_time', 'content': '04:27:33 PM'}

ChatCompletionMessage(content='The current time in New York is 4:27 PM.', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None)


In [27]:
system_message = """
You are expert in PostgreSQL queries. Given an input question, create a syntactically correct SQL query AND use the query_db function to execute the query.

 PostgreSQL "recommend_recipe_by_description" table has properties:
    #
    #  in_recipedescription character varying "input recipe description"
    #  out_recipename character varying "similar recipe name"
    #  out_similarityscore real "similarity score between 0 and 1"
    #  

        
Examples:
Question: What recipes are similar to 'chicken and peanut'?

Query:
SELECT in_recipedescription, out_recipename, out_similarityscore 
FROM recommend_recipe_by_description('chicken and peanut', 3)
WHERE out_similarityscore !=0
ORDER BY 2 DESC
; 

Answer:
The top 3 most similar recipes to 'chicken and peanut' are in markdown format: 
markdown table example:
| Recipe Name | Similarity Score |
| ------------- | ----------------- |
| Recipe 1 | 0.95 |
| Recipe 2 | 0.90 |
| Recipe 3 | 0.85 |


    
If the user is asking you for data that is not in the table, you should answer with "Error: <description of the error>":
	
Follow these Instructions:
- Generate a valid SQL query to execute on a postgreSQL database.
- Order results by most similar recipe name.
- **MUST** Call the query_db function to execute the query.
- **MUST** return the results in a markdown table format.

Question: 
"""

messages = [{"role": "system", "content": system_message}]

question = "I am looking for recipe that contains chicken and peanut. Can you suggest me few similar recipes?"        
messages.append({"role": "user", "content": f"{question}"})

assistant_response = run_conversation(messages, tools, available_functions)
print(assistant_response.choices[0].message.content)

Recommended Function call:
ChatCompletionMessageToolCall(id='call_4OC4SFnIkbnsxKsNCVuwBSHS', function=Function(arguments='{"query":"SELECT in_recipedescription, out_recipename, out_similarityscore \\nFROM recommend_recipe_by_description(\'chicken and peanut\', 3)\\nWHERE out_similarityscore !=0\\nORDER BY 2 DESC;"}', name='query_db'), type='function')

Output of function call:
[{"recipe": "Chicken Curry", "score": 0.95}, {"recipe": "Peanut Butter Chicken", "score": 0.9}, {"recipe": "Chicken Satay", "score": 0.85}]

Messages in second request:
{'role': 'system', 'content': '\nYou are expert in PostgreSQL queries. Given an input question, create a syntactically correct SQL query AND use the query_db function to execute the query.\n\n PostgreSQL "recommend_recipe_by_description" table has properties:\n    #\n    #  in_recipedescription character varying "input recipe description"\n    #  out_recipename character varying "similar recipe name"\n    #  out_similarityscore real "similarity sc

### 4.0 Calling multiple functions together
In some cases, you may want to string together multiple function calls to get the desired result. We modified the `run_conversation()` function above to allow multiple function calls to be made.

In [28]:
def run_multiturn_conversation(messages, tools, available_functions):
    # Step 1: send the conversation and available functions to GPT
    response = client.chat.completions.create(
        messages=messages,
        tools=tools,
        tool_choice="auto",
        model=model_name,
        temperature=0,
    )

    # Step 2: check if GPT wanted to call a function
    while response.choices[0].finish_reason == "tool_calls":
        response_message = response.choices[0].message
        print("Recommended Function call:")
        print(response_message.tool_calls[0])
        print()

        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors

        function_name = response_message.tool_calls[0].function.name

        # verify function exists
        if function_name not in available_functions:
            return "Function " + function_name + " does not exist"
        function_to_call = available_functions[function_name]

        # verify function has correct number of arguments
        function_args = json.loads(response_message.tool_calls[0].function.arguments)
        if check_args(function_to_call, function_args) is False:
            return "Invalid number of arguments for function: " + function_name
        function_response = function_to_call(**function_args)

        print("Output of function call:")
        print(function_response)
        print()

        # Step 4: send the info on the function call and function response to GPT

        # adding assistant response to messages
        messages.append(
            {
                "role": response_message.role,
                "function_call": {
                    "name": response_message.tool_calls[0].function.name,
                    "arguments": response_message.tool_calls[0].function.arguments,
                },
                "content": None,
            }
        )

        # adding function response to messages
        messages.append(
            {
                "role": "function",
                "name": function_name,
                "content": function_response,
            }
        )  # extend conversation with function response

        print("Messages in next request:")
        for message in messages:
            print(message)
        print()

        response = client.chat.completions.create(
            messages=messages,
            tools=tools,
            tool_choice="auto",
            model=model_name,
            temperature=0,
        )  # get a new response from GPT where it can see the function response

    return response

In [29]:
# Can add system prompting to guide the model to call functions and perform in specific ways
next_messages = [
    {
        "role": "system",
        "content": "Assistant is a helpful assistant that helps users get answers to questions. Assistant has access to several tools and sometimes you may need to call multiple tools in sequence to get answers for your users.",
    }
]
next_messages.append(
    {
        "role": "user",
        "content": "How much did S&P 500 change between July 12 and July 13? Use the calculator.",
    }
)

assistant_response = run_multiturn_conversation(
    next_messages, tools, available_functions
)
print("Final Response:")
print(assistant_response.choices[0].message.content)
print("Conversation complete!")

Recommended Function call:
ChatCompletionMessageToolCall(id='call_RBWD86RbqziEIrASYyR8GXCD', function=Function(arguments='{"index": "S&P 500"}', name='get_stock_market_data'), type='function')

Output of function call:
{"Date": {"0": "2023-07-12", "1": "2023-07-13"}, "Open": {"0": 4300.25, "1": 4325.55}, "High": {"0": 4350.32, "1": 4350.0}, "Low": {"0": 4200.2, "1": 4300.98}, "Close": {"0": 4325.74, "1": 4310.33}, "Volume": {"0": 3500000, "1": 3600000}}

Messages in next request:
{'role': 'system', 'content': 'Assistant is a helpful assistant that helps users get answers to questions. Assistant has access to several tools and sometimes you may need to call multiple tools in sequence to get answers for your users.'}
{'role': 'user', 'content': 'How much did S&P 500 change between July 12 and July 13? Use the calculator.'}
{'role': 'assistant', 'function_call': {'name': 'get_stock_market_data', 'arguments': '{"index": "S&P 500"}'}, 'content': None}
{'role': 'function', 'name': 'get_stock